In [1]:
import cramer_shoup as cs
from web3 import Web3
from solcx import compile_files
from py_ecc import bn128
import secrets

In [2]:
private_key, public_key = cs.generate_keypair()

In [120]:
#if not run before then run once
import solcx
solcx.install_solc('0.8.6')

Version('0.8.6')

In [13]:
w3 = Web3(Web3.HTTPProvider("http://localhost:8545"))
compiled_sol = compile_files(["cramer_shoup.sol"], output_values=['abi', 'bin'], solc_version="0.8.6")['cramer_shoup.sol:CramerShoup']
CramerShoup = w3.eth.contract(address='ADDRESS_OF_THE_CONTRACT', abi=compiled_sol['abi'])

In [18]:
tx_hash = CramerShoup.functions.setPublicKey(
    _pk = {
        "g1" : cs.point_to_bytes(public_key['g1']),
        "g2" : cs.point_to_bytes(public_key['g2']),
        "c" : cs.point_to_bytes(public_key['c']),
        "d" : cs.point_to_bytes(public_key['d']),
        "h" : cs.point_to_bytes(public_key['h'])
        }).transact({'from': w3.eth.accounts[0]})

tx_receipt_set_public_key = w3.eth.waitForTransactionReceipt(tx_hash)

In [5]:
b = 1 # commitment bit
n = 28 # number of bytes in the random x value
pad = 4 # number of bytes for padding

In [6]:
x = secrets.token_bytes(n)
y = cs.f(b, x)

c_b = cs.encrypt(public_key, x, pad)
c_1_b = cs.encrypt(public_key,  b'\x00' * n, pad)
r_b = c_b[-1]

c = [c_b, c_1_b] if b == 0 else [c_1_b, c_b]

In [19]:
tx_hash = CramerShoup.functions.commit(
    _com = {
        "y" : y,
        "ct0" : {
            "u1" : cs.point_to_bytes(c[0][0]),
            "u2" : cs.point_to_bytes(c[0][1]),
            "e" :  cs.point_to_bytes(c[0][2]),
            "v" : cs.point_to_bytes(c[0][3])
        },
        "ct1" : {
            "u1" : cs.point_to_bytes(c[1][0]),
            "u2" : cs.point_to_bytes(c[1][1]),
            "e" :  cs.point_to_bytes(c[1][2]),
            "v" : cs.point_to_bytes(c[1][3])
        }
    }
).transact({'from': w3.eth.accounts[0]})

tx_receipt_commit = w3.eth.waitForTransactionReceipt(tx_hash)

In [20]:
tx_hash = CramerShoup.functions.reveal(
    b = b,
    x = b'\x00' * (32 - len(x)) + x,
    k = "0x" + r_b.to_bytes(32, 'big').hex(),
    pad = pad
).transact({'from': w3.eth.accounts[0]})
tx_receipt_reveal = w3.eth.waitForTransactionReceipt(tx_hash)

In [21]:
tx_receipt_set_public_key.gasUsed

55732

In [22]:
tx_receipt_commit.gasUsed

79924

In [23]:
tx_receipt_reveal.gasUsed

129740